In [0]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import RandomOverSampler


trainCSV = "https://raw.githubusercontent.com/andybbruno/DataMining/master/new_train_cleaned.csv?token=AGWKQX6PGJXV4U2SBDZHALC6BY3P4"

In [6]:
# READ AND DROP COLUMNS
df = pd.read_csv(trainCSV)
df.drop(df.columns[0], axis=1, inplace=True)
y = df['IsBadBuy']
X = df.drop(columns=['IsBadBuy'])

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Set the parameters by cross-validation
params = {
    'n_estimators': [5, 10, 25, 50, 100],
    'learning_rate' : [0.1, 0.25, 0.5, 0.75, 1],
 }

new_params = {'adaboostclassifier__' + key: params[key] for key in params}

#scores = ['precision', 'recall', 'f1']
scores = ['recall', 'f1']

for score in scores:
    print("# Tuning hyper-parameters for ----> %s" % score)
    print()
    
    obj = make_pipeline(RandomOverSampler(), AdaBoostClassifier())
    
    if (score == "recall"):
      clf = GridSearchCV(obj, param_grid=new_params, cv=5, scoring=score)
    else:
      clf = GridSearchCV(obj, param_grid=new_params, cv=5, scoring='%s_macro' % score)
                         
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for ----> recall

Best parameters set found on development set:

{'adaboostclassifier__learning_rate': 1, 'adaboostclassifier__n_estimators': 50}

Grid scores on development set:

0.258 (+/-0.033) for {'adaboostclassifier__learning_rate': 0.1, 'adaboostclassifier__n_estimators': 5}
0.258 (+/-0.033) for {'adaboostclassifier__learning_rate': 0.1, 'adaboostclassifier__n_estimators': 10}
0.433 (+/-0.011) for {'adaboostclassifier__learning_rate': 0.1, 'adaboostclassifier__n_estimators': 25}
0.503 (+/-0.059) for {'adaboostclassifier__learning_rate': 0.1, 'adaboostclassifier__n_estimators': 50}
0.543 (+/-0.049) for {'adaboostclassifier__learning_rate': 0.1, 'adaboostclassifier__n_estimators': 100}
0.292 (+/-0.161) for {'adaboostclassifier__learning_rate': 0.25, 'adaboostclassifier__n_estimators': 5}
0.480 (+/-0.077) for {'adaboostclassifier__learning_rate': 0.25, 'adaboostclassifier__n_estimators': 10}
0.524 (+/-0.046) for {'adaboostclassifier__learning_rate': 0.25, 